In [30]:
import pandas as pd
import numpy as np
from subprocess import check_output
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import Imputer
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [31]:
X = pd.read_csv('Clustered Data');
X.head()

,Power Outage,PRCP,Delta T Min,Delta T Max,TMAX,TMIN,Fastest 5 second wind speed,5 second wind speed squared,Fastest 2 minute wind speed,2 min wind speed squared,Average Wind Speed,Avg Wind Speed Squared,Fog/Ice,Heavy/Freezing Fog,Thunder,Smoke/Haze
0,0.0,0.000,0.000000,0.000000,0.471698,0.451417,0.076216,0.005809,0.121687,0.014808,0.191874,0.036815,0.0,0.0,0.0,0.0
1,0.0,0.000,0.000054,0.000886,0.487421,0.447368,0.140000,0.019600,0.239759,0.057484,0.393679,0.154984,0.0,0.0,0.0,0.0
2,0.0,0.000,0.001362,0.000016,0.473795,0.471660,0.107568,0.011571,0.193976,0.037627,0.323251,0.104491,0.0,0.0,0.0,0.0
3,0.0,0.005,0.012255,0.000319,0.481132,0.390688,0.059459,0.003535,0.097590,0.009524,0.141309,0.019968,1.0,0.0,0.0,0.0
4,0.0,0.000,0.002301,0.000016,0.473795,0.477733,0.113514,0.012885,0.204819,0.041951,0.222122,0.049338,0.0,0.0,0.0,0.0


In [32]:
y= X[['Power Outage']];
X = X.drop(['Power Outage'], axis=1);

In [33]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

class ReduceVIF(BaseEstimator, TransformerMixin):
    def __init__(self, thresh=5.0, impute=True, impute_strategy='median'):
        # From looking at documentation, values between 5 and 10 are "okay".
        # Above 10 is too high and so should be removed.
        self.thresh = thresh
        
        # The statsmodel function will fail with NaN values, as such we have to impute them.
        # By default we impute using the median value.
        # This imputation could be taken out and added as part of an sklearn Pipeline.
        if impute:
            self.imputer = Imputer(strategy=impute_strategy)

    def fit(self, X, y=None):
        print('ReduceVIF fit')
        if hasattr(self, 'imputer'):
            self.imputer.fit(X)
        return self

    def transform(self, X, y=None):
        print('ReduceVIF transform')
        columns = X.columns.tolist()
        if hasattr(self, 'imputer'):
            X = pd.DataFrame(self.imputer.transform(X), columns=columns)
        return ReduceVIF.calculate_vif(X, self.thresh)

    @staticmethod
    def calculate_vif(X, thresh=5.0):
        # Taken from https://stats.stackexchange.com/a/253620/53565 and modified
        dropped=True
        while dropped:
            variables = X.columns
            dropped = False
            vif = [variance_inflation_factor(X[variables].values, X.columns.get_loc(var)) for var in X.columns]
            
            max_vif = max(vif)
            if max_vif > thresh:
                maxloc = vif.index(max_vif)
                print(f'Dropping {X.columns[maxloc]} with vif={max_vif}')
                X = X.drop([X.columns.tolist()[maxloc]], axis=1)
                dropped=True
        return X

In [34]:
transformer = ReduceVIF()
X= transformer.fit_transform(X,y);
X.head()

ReduceVIF fit
ReduceVIF transform
Dropping Fastest 2 minute wind speed with vif=116.40926312615994
Dropping Average Wind Speed with vif=41.15475553558485
Dropping Fastest 5 second wind speed with vif=16.96519189561377
Dropping TMIN with vif=11.935424694979043


,PRCP,Delta T Min,Delta T Max,TMAX,5 second wind speed squared,2 min wind speed squared,Avg Wind Speed Squared,Fog/Ice,Heavy/Freezing Fog,Thunder,Smoke/Haze
0,0.000,0.000000,0.000000,0.471698,0.005809,0.014808,0.036815,0.0,0.0,0.0,0.0
1,0.000,0.000054,0.000886,0.487421,0.019600,0.057484,0.154984,0.0,0.0,0.0,0.0
2,0.000,0.001362,0.000016,0.473795,0.011571,0.037627,0.104491,0.0,0.0,0.0,0.0
3,0.005,0.012255,0.000319,0.481132,0.003535,0.009524,0.019968,1.0,0.0,0.0,0.0
4,0.000,0.002301,0.000016,0.473795,0.012885,0.041951,0.049338,0.0,0.0,0.0,0.0


In [35]:
finalData = pd.concat([X,y], axis=1);
finalData.to_csv('Final Data', index=False);

In [36]:
X_new = SelectKBest(chi2, k=5).fit_transform(X.values,y.values)
X = pd.DataFrame(X_new);

In [37]:
X.head()

,0,1,2,3,4
0,0.000000,0.000000,0.036815,0.0,0.0
1,0.000054,0.000886,0.154984,0.0,0.0
2,0.001362,0.000016,0.104491,0.0,0.0
3,0.012255,0.000319,0.019968,1.0,0.0
4,0.002301,0.000016,0.049338,0.0,0.0


In [38]:
finalData.head()

,PRCP,Delta T Min,Delta T Max,TMAX,5 second wind speed squared,2 min wind speed squared,Avg Wind Speed Squared,Fog/Ice,Heavy/Freezing Fog,Thunder,Smoke/Haze,Power Outage
0,0.000,0.000000,0.000000,0.471698,0.005809,0.014808,0.036815,0.0,0.0,0.0,0.0,0.0
1,0.000,0.000054,0.000886,0.487421,0.019600,0.057484,0.154984,0.0,0.0,0.0,0.0,0.0
2,0.000,0.001362,0.000016,0.473795,0.011571,0.037627,0.104491,0.0,0.0,0.0,0.0,0.0
3,0.005,0.012255,0.000319,0.481132,0.003535,0.009524,0.019968,1.0,0.0,0.0,0.0,0.0
4,0.000,0.002301,0.000016,0.473795,0.012885,0.041951,0.049338,0.0,0.0,0.0,0.0,0.0
